In [1]:
import pandas as pd

In [2]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

## Add departement column

In [3]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [4]:
stocks.head(15)

,raison_sociale,rubric_id,quantite,unite
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq
1,GAEC PORC D'ALGRES,3660,860.0,NaN
2,GAEC PORC D'ALGRES,2102,860.0,NaN
3,GAEC FERME H ET L,2101,105.0,NaN
4,GAEC FERME H ET L,2101,150.0,NaN
5,GAEC FERME H ET L,2101,210.0,u
6,BINOUDIERE (GAEC DE LA),2102,498.0,Animaux-Eq
7,BINOUDIERE (GAEC DE LA),3660,498.0,NaN
8,BINOUDIERE (GAEC DE LA),2101,60.0,u
9,BINOUDIERE (GAEC DE LA),2101,80.0,u


In [5]:
# fonction pour avoir la dernière rubrique d'une ferme et avoir son nombre d'animaux déclaré le plus récent
def getNbAnimalsOfFarm(rub_id : int, name : str, dfD) -> int:
    dfD = dfD[(dfD.name == name) & (dfD.rubric_id == rub_id)] 
    if len(dfD.rubric_id) == 0:
        return dfD.at[0, 'quantite']
    else:
        return dfD['quantite'].iloc[-1]

In [96]:
# function pour avoir l'id de la rubrique selon le type de la ferme
def getIdRub(type: str) -> str:
    if type == 'BOVINS':
        return '2101'
    elif type == 'PORCS':
        return '2102'
    else :
        return '2111'

In [131]:
# réponse question 3
df = farms.merge(stocks, 'inner', left_on= 'name', right_on= 'raison_sociale')
df = df[['name', 'departement', 'type' ,'rubric_id', 'quantite', 'unite']]
df = df.convert_dtypes()
df = df.dropna()
dfT = df[df.departement == 44]
df = df.sort_values(by=['departement'], ascending=False)
#print(dfT.head())


def getBiggerBreedingDep(dep : int) -> str:
    nbBovins :int = 0
    nbPorcs :int = 0
    nbVolailles :int = 0
    nbTmp : int = 0
    dfDep = df[df.departement == dep].reset_index()
    name = ''
    
    for i in dfDep.index:
        if dfDep.loc[i, 'rubric_id'] == '2111' or dfDep.loc[i, 'rubric_id'] == '2101' or dfDep.loc[i, 'rubric_id'] == '2102':
            if i == 0:
                name = dfDep.loc[i, 'name']
                nbTmp = getNbAnimalsOfFarm(getIdRub(dfDep.loc[i, 'type']), name, dfDep)
            elif name != dfDep.loc[i, 'name']:
                name = dfDep.loc[i, 'name']
                nbTmp = getNbAnimalsOfFarm(getIdRub(dfDep.loc[i, 'type']), name, dfDep)       
            
            if dfDep.loc[i, 'unite'] == 'u' or dfDep.loc[i, 'unite'] == 'Animaux-Eq': 
                if dfDep.loc[i, 'type'] == 'BOVINS':
                    nbBovins += nbTmp
                elif dfDep.loc[i, 'type'] == 'PORCS': 
                    if dfDep.loc[i, 'unite'] == 'u':
                        nbPorcs += nbTmp * 0.4
                    else:
                        nbPorcs += nbTmp
                elif dfDep.loc[i, 'unite'] == 'u':
                    nbVolailles += nbTmp * 0.01 
                else:
                    nbVolailles += nbTmp      
                nbTmp = 0

    print("porc", nbPorcs)
    print("bovins", nbBovins)
    print("volaille", nbVolailles)

    if nbBovins == 0 and nbPorcs == 0 and nbVolailles == 0:
        return 'Aucune ferme dans le departement' 
    else:   
        if max(nbBovins, nbPorcs, nbVolailles) == nbBovins:
            return 'Bovin'
        elif max(nbBovins, nbPorcs, nbVolailles) == nbPorcs:
            return 'Porc'
        else:
            return 'Volaille'

# marche pas pour dep 42-44 et 72,71 et certainement d'autre
print(getBiggerBreedingDep(75))
    
reponse3 = pd.DataFrame({'departement' : [], 'reponse' : []})

reponse3.head()

porc 0
bovins 0
volaille 0
Aucune ferme dans le departement


,departement,reponse
